1) What is the approximate depth of a decision tree trained (without restrictions) on a
training set with one million instances?

>The depth of a well-balanced binary tree containing m leaves is equal to log₂(m), rounded up. log₂ is the binary log; log₂(m) = log(m) / log(2). A binary Decision Tree (one that makes only binary decisions, as is the case with all trees in Scikit-Learn) will end up more or less well balanced at the end of training, with one leaf per training instance if it is trained without restrictions. Thus, if the training set contains one million instances, the Decision Tree will have a depth of log₂(106) ≈ 20 (actually a bit more since the tree will generally not be perfectly well balanced).

2) Is a node’s Gini impurity generally lower or higher than its parent’s? Is it generally
lower/higher, or always lower/higher?

> A node's Gini impurity is generally lower than its parent's. This is due to the CART training algorithm's cost function, which splits each node in a way that minimizes the weighted sum of its children's Gini impurities. However, it is possible for a node to have a higher Gini impurity than its parent, as long as this increase is more than compensated for by a decrease in the other child's impurity. For example, consider a node containing four instances of class A and one of class B. Its Gini impurity is 1 – (1/5)² – (4/5)² = 0.32. Now suppose the dataset is one-dimensional and the instances are lined up in the following order: A, B, A, A, A. You can verify that the algorithm will split this node after the second instance, producing one child node with instances A, B, and the other child node with instances A, A, A. The first child node's Gini impurity is 1 – (1/2)² – (1/2)² = 0.5, which is higher than its parent's. This is compensated for by the fact that the other node is pure, so its overall weighted Gini impurity is 2/5 × 0.5 + 3/5 × 0 = 0.2, which is lower than the parent's Gini impurity.

3) If a decision tree is overfitting the training set, is it a good idea to try decreasing
max_depth?

>If a Decision Tree is overfitting the training set, it may be a good idea to decrease max_depth, since this will constrain the model, regularizing it.

4) If a decision tree is underfitting the training set, is it a good idea to try scaling the
input features?

>Decision Trees don't care whether or not the training data is scaled or centered; that's one of the nice things about them. So if a Decision Tree underfits the training set, scaling the input features will just be a waste of time.

5)  If it takes one hour to train a decision tree on a training set containing one million
instances, roughly how much time will it take to train another decision tree on a
training set containing ten million instances? Hint: consider the CART algorithm’s
computational complexity.

>The computational complexity of training a Decision Tree is O(n × m log₂(m)). So if you multiply the training set size by 10, the training time will be multiplied by K = (n × 10 m × log₂(10 m)) / (n × m × log₂(m)) = 10 × log₂(10 m) / log₂(m). If m = 106, then K ≈ 11.7, so you can expect the training time to be roughly 11.7 hours.

6) If it takes one hour to train a decision tree on a given training set, roughly how
much time will it take if you double the number of features?


>If the number of features doubles, then the training time will also roughly double.


7) Train and fine-tune a decision tree for the moons dataset by following these steps:
>a. Use make_moons(n_samples=10000, noise=0.4) to generate a moons
dataset.<br>
>b. Use train_test_split() to split the dataset into a training set and a test set.<br>
>c. Use grid search with cross-validation (with the help of the GridSearchCV
class) to find good hyperparameter values for a DecisionTreeClassifier.
Hint: try various values for max_leaf_nodes.<br>
>d. Train it on the full training set using these hyperparameters, and measure
your model’s performance on the test set. You should get roughly 85% to
87% accuracy.


In [1]:
#Prepare data:
from sklearn.datasets import make_moons
X_moons,y_moons = make_moons(n_samples=1000,noise=0.4)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_moons,y_moons,test_size=0.2,
                                                 random_state=42)


from sklearn.model_selection import GridSearchCV

params = {
    "max_leaf_nodes": list(range(2,100)),
    "max_depth": list(range(1,7)),
    "min_samples_split":[2,3,4]
}

from sklearn.tree import DecisionTreeClassifier
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42),
                              params,
                              cv=3)

grid_search_cv.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]})

In [2]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_depth=5, max_leaf_nodes=7, random_state=42)

In [3]:
from sklearn.metrics import precision_score,recall_score

y_pred = grid_search_cv.predict(X_test)
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))


0.8
0.8539325842696629


8) Grow a forest by following these steps:
<br>
<br>
a. Continuing the previous exercise, generate 1,000 subsets of the training set,
each containing 100 instances selected randomly. Hint: you can use ScikitLearn’s ShuffleSplit class for this.
<br>
<br>
b. Train one decision tree on each subset, using the best hyperparameter values
found in the previous exercise. Evaluate these 1,000 decision trees on the test
set. Since they were trained on smaller sets, these decision trees will likely
perform worse than the first decision tree, achieving only about 80%
accuracy.
<br>
<br>
c. Now comes the magic. For each test set instance, generate the predictions of
the 1,000 decision trees, and keep only the most frequent prediction (you can
use SciPy’s mode() function for this). This approach gives you majority-vote
predictions over the test set.
<br><br>
d. Evaluate these predictions on the test set: you should obtain a slightly higher
accuracy than your first model (about 0.5 to 1.5% higher). Congratulations,
you have trained a random forest classifier!

In [4]:
from sklearn.model_selection import ShuffleSplit
n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees,test_size = len(X_train)-n_instances,
                  random_state=42)

for mini_train_index,mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train,y_mini_train))

In [7]:
from sklearn.base import clone
from sklearn.metrics import accuracy_score

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree,(X_mini_train,y_mini_train) in zip(forest,mini_sets):
    tree.fit(X_mini_train,y_mini_train)

    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test,y_pred))



import numpy as np
np.mean(accuracy_scores)


0.810655

In [11]:
Y_pred  = np.empty([n_trees,len(X_test)],dtype=np.uint8)

for tree_index,tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

from scipy.stats import mode
y_pred_majority_votes,n_votes = mode(Y_pred,axis=0)

In [12]:
accuracy_score(y_test,y_pred_majority_votes.reshape([-1]))

0.845